# Building a Multivariate Dataset From Your Model Objects

The purpose of this Notebook is to use the Disease Propogation Objects to build a dataset testing variations in the mitigation methods. The output of this Notebook will be a csv file that contains the results from multple runs of the various interactions. 

## Steps:
1. Add the Person, Population, and Community Objects
2. Create a function that allows the Community object interventions parameters to be tested over the range of 0 to that number (Ex. Population Size, Number of Interactions, Percentage Masked). **NOTE: make sure to add any custom interventions you may have added.**
3. Alter the code to ouput to a csv file
4. Run the simulation to create your dataset.


## Step 1. Add our Person, Population, and Community Class Objects

#### Person Object

In [1]:
import numpy as np
import random

# Person Object AKA Joe
class Person(object):
    def __init__(self):
        self.status = "Susceptible"
        self.days_sick = 0
        self.sickness_rate = 10
        self.Interaction = []
        self.masked = False
        
    def maskUp(self, wears_mask):
        if (wears_mask):
            self.set_sickness_rate(self.sickness_rate/2)
            self.masked = True
    
    def set_sickness_rate(self, new_rate):
        self.sickness_rate = new_rate
    
    def infect(self, number_of_days_sick):
        self.status = "Infected"
        self.days_sick = number_of_days_sick
    
    def recover(self):
        self.status = "Recovered"
        self.days_sick = 0
    
    def update(self):
        if (self.status == "Susceptible"):
            self.roll_for_infection()
            
        elif (self.status == "Infected"):            
            self.days_sick = self.days_sick - 1
            if (self.days_sick <= 0):
                self.recover()
    
    def update_days_sick(self):
        if (self.status == "Infected"):            
            self.days_sick = self.days_sick - 1
            if (self.days_sick <= 0):
                self.recover()
            

            
    def roll_for_infection(self):
        is_sick = (random.randint(0,101) < self.sickness_rate)
        if (is_sick):
            self.infect(4)

#### Population Community

In [2]:
# Population Object
class Population:
    def __init__(self, number=100, interaction=10, percentage_masked=0):
        self.People = []
        self.number = number
        self.interaction = interaction
        self.percentage_masked = percentage_masked
        
        if (number < 100):
            self.number = 100
        
        if (interaction < 10):
            self.interaction = 10
        
        ##build our population
        patient_Zero = Person()
        patient_Zero.infect(5)
        
        self.People.append(patient_Zero)
        
        for i in range(0,number):
            p = Person()
            if (i < number*(self.percentage_masked/100.0) and self.percentage_masked>0):
                p.maskUp(True)
                
            self.People.append(p)

               
            
    def number_of_sick(self):
        sick_Count = 0
        for j in self.People:
            if (j.status == "Infected"):
                sick_Count = sick_Count + 1
                
        return sick_Count
    
    def number_of_recovered(self):
        recovered_Count = 0
        for j in self.People:
            if (j.status == "Recovered"):
                recovered_Count = recovered_Count + 1
        
        return recovered_Count
    
    def number_of_susceptible(self):
        susceptible_Count = 0
        for j in self.People:
            if (j.status == "Susceptible"):
                susceptible_Count = susceptible_Count + 1
                
        return susceptible_Count
    
    def number_masked(self):
        masked_Count = 0
        for j in self.People:
            if (j.masked):
                masked_Count = masked_Count + 1
        
        return masked_Count
    
    def number_masked_sick(self):
        masked_SickCount = 0
        for j in self.People:
            if (j.masked and j.status == "Infected"):
                masked_SickCount = masked_SickCount + 1
        
        return masked_SickCount
    
    ## Here's the brains of the Population class (and our code)
    ## the update will be ran everyday (meaning, every iteration of our while loop)
    ## build interaction tables (who interacted with whom)
    ## if marked sick they must roll to see if (one per interaction)
    ## if I interacted with 10 people, each of them must roll and vice versa if they
    ## they were marked as infected
    def update(self):
        ## people who are might get sick
        might_Get_Sick = []
        
        for person in self.People:
            
            #clear out your interactions per day
            person.Interaction = []
            
            ##These are the n number of people that you interact with
            for i in range(0, self.interaction):
                r = random.randint(0,self.number)
                person.Interaction.append(r)
                
            ##if "person" is Infected, every single person they interacted with might get sick
            if (person.status == "Infected"):
                person.update_days_sick()
                
                for m in person.Interaction:
                    if (self.People[m].status == "Susceptible"): #only susceptible people can get sick
                        might_Get_Sick.append(self.People[m])
            
            ##if "person" is susceptible and they interact w/ someone who is "infected" 
            ##person gets added to the "might get sick"
            if (person.status == "Susceptible"):
                for m in person.Interaction:
                    if (self.People[m].status == "Infected"):
                        might_Get_Sick.append(person)
            
        ##Now! We're going to roll through our "Might Get Sick" and they each roll a die
        ##this is only a list of susceptible people
        for p in might_Get_Sick:
            p.roll_for_infection()

#### Person Community

In [3]:
# Define the Community
# Population( size_of_population, number_of_interactions, percentage_wearing_masks)
myCommunity = Population(8000,10,50)
day = 0
my_plot = []
number_sick = myCommunity.number_of_sick()
while (number_sick > 0):
    number_sick = myCommunity.number_of_sick()
    number_recovered = myCommunity.number_of_recovered()
    number_masked = myCommunity.number_masked()
    number_masked_sick = myCommunity.number_masked_sick()
    
    my_plot.append([day, number_sick, number_recovered])
    
    if (myCommunity.percentage_masked > 0):
        print ("day:", day, "Number of sick:", number_sick, "Hosptalized:", number_sick*(.04), "Sick Mask Wearers:", number_masked_sick, "Mask Wearers:", number_masked)
    else:
        print ("day:", day, "Number of sick:", number_sick)
    
    day = day+1
    myCommunity.update()

#print ("day:", day, "Number of sick:", myCommunity.number_of_sick())
print ("Number recovered:", myCommunity.number_of_recovered())
print ("Number susceptible:", myCommunity.number_of_susceptible())

day: 0 Number of sick: 1 Hosptalized: 0.04 Sick Mask Wearers: 0 Mask Wearers: 4000
day: 1 Number of sick: 6 Hosptalized: 0.24 Sick Mask Wearers: 1 Mask Wearers: 4000
day: 2 Number of sick: 14 Hosptalized: 0.56 Sick Mask Wearers: 5 Mask Wearers: 4000
day: 3 Number of sick: 41 Hosptalized: 1.6400000000000001 Sick Mask Wearers: 13 Mask Wearers: 4000
day: 4 Number of sick: 92 Hosptalized: 3.68 Sick Mask Wearers: 35 Mask Wearers: 4000
day: 5 Number of sick: 224 Hosptalized: 8.96 Sick Mask Wearers: 72 Mask Wearers: 4000
day: 6 Number of sick: 549 Hosptalized: 21.96 Sick Mask Wearers: 184 Mask Wearers: 4000
day: 7 Number of sick: 1236 Hosptalized: 49.44 Sick Mask Wearers: 438 Mask Wearers: 4000
day: 8 Number of sick: 2496 Hosptalized: 99.84 Sick Mask Wearers: 929 Mask Wearers: 4000
day: 9 Number of sick: 4243 Hosptalized: 169.72 Sick Mask Wearers: 1705 Mask Wearers: 4000
day: 10 Number of sick: 5644 Hosptalized: 225.76 Sick Mask Wearers: 2493 Mask Wearers: 4000
day: 11 Number of sick: 5911 Ho

# 2. Make a function that allows the interventions to be easily tested

Create a function that allows the Community object interventions parameters to be tested over the range of 0 to that number (Ex. Population Size, Number of Interactions, Percentage Masked). 

In [5]:
def myCommunityTests(pop_test, interactions_test, masked_test):
    # Population( size_of_population, number_of_interactions, percentage_wearing_masks)
    myCommunity = Population(pop_test,interactions_test,masked_test)
    day = 0
    my_plot = []
    number_sick = myCommunity.number_of_sick()
    while (number_sick > 0):
        number_sick = myCommunity.number_of_sick()
        number_recovered = myCommunity.number_of_recovered()
        number_masked = myCommunity.number_masked()
        number_masked_sick = myCommunity.number_masked_sick()
    
        my_plot.append([day, number_sick, number_recovered])
    
        if (myCommunity.percentage_masked > 0):
            print ("day:", day, "Number of sick:", number_sick, "Hosptalized:", number_sick*(.04), "Sick Mask Wearers:", number_masked_sick, "Mask Wearers:", number_masked)
        else:
            print ("day:", day, "Number of sick:", number_sick)
    
        day = day+1
        myCommunity.update()

    #print ("day:", day, "Number of sick:", myCommunity.number_of_sick())
    print ("Number recovered:", myCommunity.number_of_recovered())
    print ("Number susceptible:", myCommunity.number_of_susceptible())

In [6]:
myCommunityTests(500, 3, 40)

day: 0 Number of sick: 1 Hosptalized: 0.04 Sick Mask Wearers: 0 Mask Wearers: 200
day: 1 Number of sick: 3 Hosptalized: 0.12 Sick Mask Wearers: 0 Mask Wearers: 200
day: 2 Number of sick: 9 Hosptalized: 0.36 Sick Mask Wearers: 0 Mask Wearers: 200
day: 3 Number of sick: 28 Hosptalized: 1.12 Sick Mask Wearers: 6 Mask Wearers: 200
day: 4 Number of sick: 63 Hosptalized: 2.52 Sick Mask Wearers: 12 Mask Wearers: 200
day: 5 Number of sick: 134 Hosptalized: 5.36 Sick Mask Wearers: 32 Mask Wearers: 200
day: 6 Number of sick: 246 Hosptalized: 9.84 Sick Mask Wearers: 69 Mask Wearers: 200
day: 7 Number of sick: 340 Hosptalized: 13.6 Sick Mask Wearers: 108 Mask Wearers: 200
day: 8 Number of sick: 376 Hosptalized: 15.040000000000001 Sick Mask Wearers: 142 Mask Wearers: 200
day: 9 Number of sick: 337 Hosptalized: 13.48 Sick Mask Wearers: 144 Mask Wearers: 200
day: 10 Number of sick: 229 Hosptalized: 9.16 Sick Mask Wearers: 114 Mask Wearers: 200
day: 11 Number of sick: 120 Hosptalized: 4.8 Sick Mask We

## 3. Alter the code to ouput to a csv file

In [36]:
# Alter the myCommunityTests function to ouput wanted data to a csv

#Ex: Population_Size, Number_Interactions, Percentage_Masked, Total_Days, Number_Recovered, Number_Suceptible, Max_Sick, Max_Hospitalized
import pandas as pd

def myCommunityTests_toDict(pop_test, interactions_test, masked_test, hospitalization_rate = .04):
    # Population( size_of_population, number_of_interactions, percentage_wearing_masks)
    myCommunity = Population(pop_test,interactions_test,masked_test)
    day = 0
    my_plot = []
    simulation_sumary = {
        "Population_Size" : pop_test,
        "Number_Interactions" : interactions_test,
        "Percentage_Masked" : masked_test,
        "Hospitalization_Rate" : hospitalization_rate
    }
    
    number_sick = myCommunity.number_of_sick()
    while (number_sick > 0):
        number_sick = myCommunity.number_of_sick()
        number_recovered = myCommunity.number_of_recovered()
        number_masked = myCommunity.number_masked()
        number_masked_sick = myCommunity.number_masked_sick()
    
        my_plot.append([day, number_sick, number_recovered, number_sick*(hospitalization_rate), number_masked_sick, number_masked])
    
        #if (myCommunity.percentage_masked > 0):
            #print ("day:", day, "Number of sick:", number_sick, "Hosptalized:", number_sick*(hospitalization_rate), "Sick Mask Wearers:", number_masked_sick, "Mask Wearers:", number_masked)
        #else:
            #print ("day:", day, "Number of sick:", number_sick)
    
        day = day+1
        myCommunity.update()
    
    dataset = pd.DataFrame(my_plot, columns=['day','number_sick', 'number_recovered','hosptalized', 'sick_mask_wearers','mask_wearers' ])
    #print ("day:", day, "Number of sick:", myCommunity.number_of_sick())
    #print ("Number recovered:", myCommunity.number_of_recovered())
    simulation_sumary["recovered"] = myCommunity.number_of_recovered()
    #print ("Number susceptible:", myCommunity.number_of_susceptible())
    simulation_sumary["susceptible"] = myCommunity.number_of_susceptible()
    #print (dataset.head())
    simulation_sumary["total_days"] = dataset["day"].max()
    simulation_sumary["max_sick"] = dataset["number_sick"].max()
    simulation_sumary["max_hosptalized"] = dataset["hosptalized"].max()
    simulation_sumary["max_sick_mask_wearers"] = dataset["sick_mask_wearers"].max()
    simulation_sumary["total_mask_wearers"] = dataset["mask_wearers"].max()
    
    print(simulation_sumary)
    return simulation_sumary
    
    


In [37]:
myCommunityTests_toDict(5000, 5, 50)

{'Population_Size': 5000, 'Number_Interactions': 5, 'Percentage_Masked': 50, 'Hospitalization_Rate': 0.04, 'recovered': 4936, 'susceptible': 65, 'total_days': 25, 'max_sick': 3616, 'max_hosptalized': 144.64000000000001, 'max_sick_mask_wearers': 1770, 'total_mask_wearers': 2500}


{'Population_Size': 5000,
 'Number_Interactions': 5,
 'Percentage_Masked': 50,
 'Hospitalization_Rate': 0.04,
 'recovered': 4936,
 'susceptible': 65,
 'total_days': 25,
 'max_sick': 3616,
 'max_hosptalized': 144.64000000000001,
 'max_sick_mask_wearers': 1770,
 'total_mask_wearers': 2500}

# 4. Now lets run our simulation with a range of values for our interventions

In [46]:
import csv

test_population = 3000
maximum_interactions = 10
maximum_mask_percentage = 20
hospital_rate = .4

# Name of the output csv file
filename = "mycsvfile.csv"


for i in range(0, maximum_interactions, 1):
     for m in range(0, maximum_mask_percentage, 1):
        
          sim_data = myCommunityTests_toDict(test_population, i, m, hospital_rate)
          print(sim_data)
    
          if (i == 0 and m == 0):
             with open(filename, 'w') as f:  
                 w = csv.DictWriter(f, sim_data.keys())
                 w.writeheader()
          else:
              with open(filename, 'a') as f:  # You will need 'wb' mode in Python 2.x
                w = csv.DictWriter(f, sim_data.keys())
                w.writerow(sim_data)

  
        
       

{'Population_Size': 3000, 'Number_Interactions': 0, 'Percentage_Masked': 0, 'Hospitalization_Rate': 0.4, 'recovered': 3001, 'susceptible': 0, 'total_days': 16, 'max_sick': 2565, 'max_hosptalized': 1026.0, 'max_sick_mask_wearers': 0, 'total_mask_wearers': 0}
{'Population_Size': 3000, 'Number_Interactions': 0, 'Percentage_Masked': 0, 'Hospitalization_Rate': 0.4, 'recovered': 3001, 'susceptible': 0, 'total_days': 16, 'max_sick': 2565, 'max_hosptalized': 1026.0, 'max_sick_mask_wearers': 0, 'total_mask_wearers': 0}
{'Population_Size': 3000, 'Number_Interactions': 0, 'Percentage_Masked': 1, 'Hospitalization_Rate': 0.4, 'recovered': 3000, 'susceptible': 1, 'total_days': 17, 'max_sick': 2616, 'max_hosptalized': 1046.4, 'max_sick_mask_wearers': 26, 'total_mask_wearers': 30}
{'Population_Size': 3000, 'Number_Interactions': 0, 'Percentage_Masked': 1, 'Hospitalization_Rate': 0.4, 'recovered': 3000, 'susceptible': 1, 'total_days': 17, 'max_sick': 2616, 'max_hosptalized': 1046.4, 'max_sick_mask_wear